In [1]:
import os

os.chdir("/Users/teon/Projects/241028_TRADE/trade-dashboard/")
print(os.getcwd())

/Users/teon/Projects/241028_TRADE/trade-dashboard


In [2]:
import warnings

import pandas as pd  # type: ignore

from notebook.dynamodb import RecordFetcher

warnings.filterwarnings('ignore')

In [3]:
table_name = "td-profit-loss"
records = RecordFetcher.get_all_records(table_name)

In [4]:
df = pd.DataFrame(records)
df = df[df["isSettled"]]
df["enteredAt"] = pd.to_datetime(df["enteredAt"], format="%Y-%m-%d_%H-%M")
df_extract = df[df["enteredAt"].dt.month == 1]
df_extract["profitLossPrice"] = df_extract["profitLossPrice"].astype(int)
df_extract["winlose"] = df_extract["profitLossPrice"].apply(lambda x: "勝ち" if x > 0 else "負け")

In [5]:
text = "|結果|事前判断|事後反省|\n" + "|-|-|-|\n"
for i, row in df_extract.iterrows():
    text += f"|{row['winlose']}|{row['reason'].replace('\n', '')}|{row['result'].replace('\n', '')}|\n"
print(text)

|結果|事前判断|事後反省|
|-|-|-|
|負け|1期で直前の谷を更新したのでショートで入る。|その後反発上昇。入るのが早すぎた。ショートであっても、更新確認のために2期は待ったほうがいい。|
|負け|下落レンジでレジスタンスに到達したので、ショート読み。|普通に外した。|
|負け|逆トライアングルのレジスタンス付近に到達。ショート。|外した。上位のレンジから大きく逸脱していて、読めない。|
|勝ち|直前の谷の安値を大幅を更新したため、サポートが破られ、転換と判断。ショート。|予想通り転換が発生。下落のため急な値動き。その後2期で高値更新のため転換リスクを考慮して一度売り抜け。4日下落トレンドかつ上位足がまだ遠いため、次のサポートを破ったらまたショートではいる。|
|負け|上位足の上昇レンジの底に到達したため、反発狙い。|損切りが厳しすぎた。|
|負け|上位サポート反発|上位サポートが更新されてしまった|
|負け|上位のサポートに到達。ロング。|欧州へのトランプ関税の影響で、円買い。最近急落が多く負け続けてる。|
|負け|サポート突破して修正が入ったため、レジスタンス化することにかけてショート。普通にだめなこともあるので、損切り厳し目。|実需筋のドル買いによって反発上昇。やはり、レジスタンスをちゃんと確認したほうがいい。また、上位足近くで取れないと、パワー不足で乙る場合が多い。|
|負け|もう一度試す。|上位近くないのに入ったのがそもそも間違いだった|
|負け|ウェッジ。修正波と見て、ロング。|下がりすぎた。トライアングルに下向き要素がある時点でダメかも。|
|負け|ダブルトップで、かつサポートを更新。ショート。|トレンド通りになったが、謎の円安により狩られた。損切りを0円にしていたので損失は最小に抑えた。|
|負け|上昇トライアングルで、1回サポートで反発したため、サポートが機能していると見込んでロング。ただし損切りは厳しく設定。|伸び切らなかったので損切り。まだ損失が大き過ぎるのでもっと絞る。|
|負け|下落レンジでのレジスタンスがサポート化下と判断。|日銀利上げ発表による急落で下落。その後持ち直すのか？|
|勝ち|上昇トライアングルで下に突破。その後レジスタンスになったと見てエントリー。|上位近くないのに入ってしまったので一旦抜ける。|
|負け|下落レンジ

In [6]:
plus_pips_list = []
lose_times = 0
for i, row in df_extract.iterrows():
    # 差額
    if row["position"] == "long":
        diff = float(row["exitPrice"]) - float(row["entryPrice"])
    if row["position"] == "short":
        diff = float(row["entryPrice"]) - float(row["exitPrice"])
    # pips計算
    def _pips():
        match row["quoteCurrency"]:
            case "JPY":
                return diff * 100
            case "ZAR" | "MXN":
                return diff * 1000
            case _:
                return diff * 10000
    pips = _pips()
    if pips > 0:
        plus_pips_list.append(pips)
    else:
        lose_times += 1
total_plus_pips = sum(plus_pips_list)
loss_cut_pips = total_plus_pips / lose_times
print(f"損失pipsの上限値: {loss_cut_pips}")

損失pipsの上限値: 21.71708333333325
